In [1]:
import pandas as pd
import geopandas as gpd
import requests as re
from io import BytesIO
from os import path, makedirs

# Produção de habitação de interesse social

Vários indicadores do GT Urbanismo são relacionados à produção e entrega de unidades de habitação de interesse social. O formulário 15 é relacionado ao orçamento utilizado na produção de unidades de habitação de interesse social. O primeiro indicador físico está no formulário 18 e diz respeito à meta de provimento de moradias do PdM 2021-2024. O formulário 20 é relacionado a uma iniciativa específica do PdM, mas os dados de execução estão disponíveis apenas no relatório da Função Habitação. Por último, os formulários 19 e 21 são relacionado, mas não possuem meta no PdM, apenas nos ODS da Agenda 2030.

Posteriormente, um novo documento apresentou várias informações diferentes relacionadas ao orçamento da função habitação, onde está incluso o Programa 3002. Portanto, vamos manter todo o orçamento da função habitação e apresentar os diferentes recortes no Qlik Sense.

## Orçamento previsto/liquidado no Programa 3002

Vamos começar coletando os dados orçamentários do site de execução orçamentária da Secretaria da Fazenda.

In [2]:
url_orcamento = 'https://orcamento.sf.prefeitura.sp.gov.br/orcamento/uploads/2024/basedadosexecucao_1224.csv'
df_orcamento = pd.read_csv(url_orcamento,
                           sep=';',
                           decimal=',',
                           encoding='latin1',
                           dtype=str)
df_orcamento

,DataInicial,DataFinal,Cd_AnoExecucao,Cd_Exercicio,Cd_Dotacao_Id,Administracao,Cd_Orgao,Sigla_Orgao,Ds_Orgao,Cd_Unidade,...,Vl_Congelado,Vl_Descongelado,Vl_CongeladoLiquido,Disponivel,Vl_ReservadoLiquido,Vl_EmpenhadoLiquido,Vl_Liquidado,Vl_Pago,Saldo_Dotacao,DataExtracao
0,01/01/2024,31/12/2024,2024,2024,169108,Direta,07,FMD,Fundo Municipal de Desenvolvimento Social,10,...,0,0,0,1000,0,0,0,0,1000,18/01/2025
1,01/01/2024,31/12/2024,2024,2024,173723,Direta,07,FMD,Fundo Municipal de Desenvolvimento Social,10,...,0,0,0,"1317978,6",1314177,1314177,1314177,1314177,"3801,6",18/01/2025
2,01/01/2024,31/12/2024,2024,2024,171046,Direta,07,FMD,Fundo Municipal de Desenvolvimento Social,10,...,0,0,0,"3131324,4","3131324,4","3131324,4","2940879,9","2791266,85",0,18/01/2025
3,01/01/2024,31/12/2024,2024,2024,173862,Direta,07,FMD,Fundo Municipal de Desenvolvimento Social,10,...,0,0,0,"37258939,15","37222552,12","37222552,12","30999672,71","30291725,36","36387,03",18/01/2025
4,01/01/2024,31/12/2024,2024,2024,180471,Direta,07,FMD,Fundo Municipal de Desenvolvimento Social,10,...,0,0,0,2000000,2000000,2000000,2000000,2000000,0,18/01/2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9339,01/01/2024,31/12/2024,2024,2024,164854,Legislativo,77,FTCMSP,Fundo Especial de Despesas do Tribunal de Contas,10,...,0,0,0,24000,0,0,0,0,24000,18/01/2025
9340,01/01/2024,31/12/2024,2024,2024,182836,Legislativo,77,FTCMSP,Fundo Especial de Despesas do Tribunal de Contas,10,...,0,0,0,20000,1856,1856,1856,1856,18144,18/01/2025
9341,01/01/2024,31/12/2024,2024,2024,173543,Legislativo,77,FTCMSP,Fundo Especial de Despesas do Tribunal de Contas,10,...,0,0,0,400000,"201545,67","201545,67","201545,67","201545,67","198454,33",18/01/2025
9342,01/01/2024,31/12/2024,2024,2024,164855,Legislativo,77,FTCMSP,Fundo Especial de Despesas do Tribunal de Contas,10,...,0,0,0,6252,0,0,0,0,6252,18/01/2025


In [3]:
for col in [col for col in df_orcamento.columns if 'Vl' in col]:
    df_orcamento.loc[:, col] = df_orcamento[col].str.replace(',', '.').astype(float)
df_orcamento.loc[:, 'DataExtracao'] = pd.to_datetime(df_orcamento['DataExtracao'], format='%d/%m/%Y')
df_orcamento

,DataInicial,DataFinal,Cd_AnoExecucao,Cd_Exercicio,Cd_Dotacao_Id,Administracao,Cd_Orgao,Sigla_Orgao,Ds_Orgao,Cd_Unidade,...,Vl_Congelado,Vl_Descongelado,Vl_CongeladoLiquido,Disponivel,Vl_ReservadoLiquido,Vl_EmpenhadoLiquido,Vl_Liquidado,Vl_Pago,Saldo_Dotacao,DataExtracao
0,01/01/2024,31/12/2024,2024,2024,169108,Direta,07,FMD,Fundo Municipal de Desenvolvimento Social,10,...,0.0,0.0,0.0,1000,0.0,0.0,0.0,0.0,1000,2025-01-18 00:00:00
1,01/01/2024,31/12/2024,2024,2024,173723,Direta,07,FMD,Fundo Municipal de Desenvolvimento Social,10,...,0.0,0.0,0.0,"1317978,6",1314177.0,1314177.0,1314177.0,1314177.0,"3801,6",2025-01-18 00:00:00
2,01/01/2024,31/12/2024,2024,2024,171046,Direta,07,FMD,Fundo Municipal de Desenvolvimento Social,10,...,0.0,0.0,0.0,"3131324,4",3131324.4,3131324.4,2940879.9,2791266.85,0,2025-01-18 00:00:00
3,01/01/2024,31/12/2024,2024,2024,173862,Direta,07,FMD,Fundo Municipal de Desenvolvimento Social,10,...,0.0,0.0,0.0,"37258939,15",37222552.12,37222552.12,30999672.71,30291725.36,"36387,03",2025-01-18 00:00:00
4,01/01/2024,31/12/2024,2024,2024,180471,Direta,07,FMD,Fundo Municipal de Desenvolvimento Social,10,...,0.0,0.0,0.0,2000000,2000000.0,2000000.0,2000000.0,2000000.0,0,2025-01-18 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9339,01/01/2024,31/12/2024,2024,2024,164854,Legislativo,77,FTCMSP,Fundo Especial de Despesas do Tribunal de Contas,10,...,0.0,0.0,0.0,24000,0.0,0.0,0.0,0.0,24000,2025-01-18 00:00:00
9340,01/01/2024,31/12/2024,2024,2024,182836,Legislativo,77,FTCMSP,Fundo Especial de Despesas do Tribunal de Contas,10,...,0.0,0.0,0.0,20000,1856.0,1856.0,1856.0,1856.0,18144,2025-01-18 00:00:00
9341,01/01/2024,31/12/2024,2024,2024,173543,Legislativo,77,FTCMSP,Fundo Especial de Despesas do Tribunal de Contas,10,...,0.0,0.0,0.0,400000,201545.67,201545.67,201545.67,201545.67,"198454,33",2025-01-18 00:00:00
9342,01/01/2024,31/12/2024,2024,2024,164855,Legislativo,77,FTCMSP,Fundo Especial de Despesas do Tribunal de Contas,10,...,0.0,0.0,0.0,6252,0.0,0.0,0.0,0.0,6252,2025-01-18 00:00:00


In [4]:
df_orcamento = df_orcamento.loc[df_orcamento['Cd_Funcao']=='16']
df_orcamento

,DataInicial,DataFinal,Cd_AnoExecucao,Cd_Exercicio,Cd_Dotacao_Id,Administracao,Cd_Orgao,Sigla_Orgao,Ds_Orgao,Cd_Unidade,...,Vl_Congelado,Vl_Descongelado,Vl_CongeladoLiquido,Disponivel,Vl_ReservadoLiquido,Vl_EmpenhadoLiquido,Vl_Liquidado,Vl_Pago,Saldo_Dotacao,DataExtracao
27,01/01/2024,31/12/2024,2024,2024,169113,Direta,07,FMD,Fundo Municipal de Desenvolvimento Social,10,...,0.0,0.0,0.0,1000,0.0,0.0,0.0,0.0,1000,2025-01-18 00:00:00
28,01/01/2024,31/12/2024,2024,2024,167099,Direta,07,FMD,Fundo Municipal de Desenvolvimento Social,10,...,0.0,0.0,0.0,1000,0.0,0.0,0.0,0.0,1000,2025-01-18 00:00:00
29,01/01/2024,31/12/2024,2024,2024,167101,Direta,07,FMD,Fundo Municipal de Desenvolvimento Social,10,...,0.0,0.0,0.0,1000,0.0,0.0,0.0,0.0,1000,2025-01-18 00:00:00
30,01/01/2024,31/12/2024,2024,2024,171051,Direta,07,FMD,Fundo Municipal de Desenvolvimento Social,10,...,0.0,0.0,0.0,14985177,14963773.14,14963773.14,14842626.58,14828557.72,"21403,86",2025-01-18 00:00:00
31,01/01/2024,31/12/2024,2024,2024,173960,Direta,07,FMD,Fundo Municipal de Desenvolvimento Social,10,...,0.0,0.0,0.0,10000000,9968965.45,9968965.45,6827361.22,6827361.22,"31034,55",2025-01-18 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9199,01/01/2024,31/12/2024,2024,2024,182123,Indireta,91,FMH,Fundo Municipal de Habitação,10,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,2025-01-18 00:00:00
9200,01/01/2024,31/12/2024,2024,2024,182125,Indireta,91,FMH,Fundo Municipal de Habitação,10,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,2025-01-18 00:00:00
9201,01/01/2024,31/12/2024,2024,2024,164701,Indireta,91,FMH,Fundo Municipal de Habitação,10,...,2745125.68,0.0,2745125.68,"10311885,81",6264151.26,6264151.26,5565823.14,5492338.9,"4047734,55",2025-01-18 00:00:00
9202,01/01/2024,31/12/2024,2024,2024,182481,Indireta,91,FMH,Fundo Municipal de Habitação,10,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,2025-01-18 00:00:00


## Orçamento regionalizado no Programa 3002

O orçamento é regionalizado apenas na liquidação, então não é possível obter o orçamento previsto por subprefeitura, mas é possível obter o orçamento liquidado.

In [5]:
url_orcamento_r = 'https://orcamento.sf.prefeitura.sp.gov.br/orcamento/uploads/2024/basedadosDA_1224.csv'
df_orcamento_r = pd.read_csv(url_orcamento_r,
                           sep=';',
                           decimal=',',
                           encoding='latin1',
                           dtype=str)
df_orcamento_r

,COD_EMPRESA_PMSP,COD_EMPENHO,ANO_EMPENHO,CÓDIGO_NLP,ANO_LIQUIDAÇÃO,DATA_LIQUIDAÇÃO,CÓDIGO_ÓRGÃO,SIGLA_ÓRGÃO,DESCRIÇÃO_ÓRGÃO,CÓDIGO_UNIDADE,...,CÓDIGO_EXERCÍCIO_FONTE,CÓDIGO_DESTINAÇÃO_RECURSO,CÓDIGO_VÍNCULO_PMSP,CÓDIGO_TIPO_CRÉDITO_ORÇAMENTÁRIO,REGIÃO,SUBPREFEITURA,DISTRITO,TIPO_REGIONALIZAÇÃO,VALOR_DETALHAMENTO_AÇÃO,DATA_EXTRAÇÃO
0,04,197,2024,362,2024,2024-01-30 00:00:00.0000000,02,HSPM,Hospital do Servidor Público Municipal,10,...,1,500,9001,0,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,"1792,03",08/10/2025
1,04,197,2024,893,2024,2024-02-28 00:00:00.0000000,02,HSPM,Hospital do Servidor Público Municipal,10,...,1,500,9001,0,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,"1780,47",08/10/2025
2,04,197,2024,1973,2024,2024-04-05 00:00:00.0000000,02,HSPM,Hospital do Servidor Público Municipal,10,...,1,500,9001,0,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,"1667,19",08/10/2025
3,04,197,2024,2430,2024,2024-04-29 00:00:00.0000000,02,HSPM,Hospital do Servidor Público Municipal,10,...,1,500,9001,0,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,"1667,19",08/10/2025
4,04,2028,2024,3142,2024,2024-05-29 00:00:00.0000000,02,HSPM,Hospital do Servidor Público Municipal,10,...,1,500,9001,0,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,"1782,02",08/10/2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531013,01,163784,2024,361586,2024,2024-12-23 00:00:00.0000000,99,FUNDIP,Fundo Municipal de Iluminação Pública,10,...,2,751,0253,1,NaN,NaN,NaN,NaN,NaN,08/10/2025
531014,01,163784,2024,361589,2024,2024-12-23 00:00:00.0000000,99,FUNDIP,Fundo Municipal de Iluminação Pública,10,...,2,751,0253,1,NaN,NaN,NaN,NaN,NaN,08/10/2025
531015,01,163784,2024,361604,2024,2024-12-23 00:00:00.0000000,99,FUNDIP,Fundo Municipal de Iluminação Pública,10,...,2,751,0253,1,NaN,NaN,NaN,NaN,NaN,08/10/2025
531016,01,163784,2024,361606,2024,2024-12-23 00:00:00.0000000,99,FUNDIP,Fundo Municipal de Iluminação Pública,10,...,2,751,0253,1,NaN,NaN,NaN,NaN,NaN,08/10/2025


In [6]:
df_orcamento_r.loc[:, 'VALOR_DETALHAMENTO_AÇÃO'] = df_orcamento_r['VALOR_DETALHAMENTO_AÇÃO'].str.replace(',', '.').astype(float)
df_orcamento_r.loc[:, 'DATA_EXTRAÇÃO'] = pd.to_datetime(df_orcamento_r['DATA_EXTRAÇÃO'], format='%d/%m/%Y')
df_orcamento_r

,COD_EMPRESA_PMSP,COD_EMPENHO,ANO_EMPENHO,CÓDIGO_NLP,ANO_LIQUIDAÇÃO,DATA_LIQUIDAÇÃO,CÓDIGO_ÓRGÃO,SIGLA_ÓRGÃO,DESCRIÇÃO_ÓRGÃO,CÓDIGO_UNIDADE,...,CÓDIGO_EXERCÍCIO_FONTE,CÓDIGO_DESTINAÇÃO_RECURSO,CÓDIGO_VÍNCULO_PMSP,CÓDIGO_TIPO_CRÉDITO_ORÇAMENTÁRIO,REGIÃO,SUBPREFEITURA,DISTRITO,TIPO_REGIONALIZAÇÃO,VALOR_DETALHAMENTO_AÇÃO,DATA_EXTRAÇÃO
0,04,197,2024,362,2024,2024-01-30 00:00:00.0000000,02,HSPM,Hospital do Servidor Público Municipal,10,...,1,500,9001,0,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,1792.03,2025-10-08 00:00:00
1,04,197,2024,893,2024,2024-02-28 00:00:00.0000000,02,HSPM,Hospital do Servidor Público Municipal,10,...,1,500,9001,0,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,1780.47,2025-10-08 00:00:00
2,04,197,2024,1973,2024,2024-04-05 00:00:00.0000000,02,HSPM,Hospital do Servidor Público Municipal,10,...,1,500,9001,0,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,1667.19,2025-10-08 00:00:00
3,04,197,2024,2430,2024,2024-04-29 00:00:00.0000000,02,HSPM,Hospital do Servidor Público Municipal,10,...,1,500,9001,0,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,1667.19,2025-10-08 00:00:00
4,04,2028,2024,3142,2024,2024-05-29 00:00:00.0000000,02,HSPM,Hospital do Servidor Público Municipal,10,...,1,500,9001,0,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,1782.02,2025-10-08 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531013,01,163784,2024,361586,2024,2024-12-23 00:00:00.0000000,99,FUNDIP,Fundo Municipal de Iluminação Pública,10,...,2,751,0253,1,NaN,NaN,NaN,NaN,NaN,2025-10-08 00:00:00
531014,01,163784,2024,361589,2024,2024-12-23 00:00:00.0000000,99,FUNDIP,Fundo Municipal de Iluminação Pública,10,...,2,751,0253,1,NaN,NaN,NaN,NaN,NaN,2025-10-08 00:00:00
531015,01,163784,2024,361604,2024,2024-12-23 00:00:00.0000000,99,FUNDIP,Fundo Municipal de Iluminação Pública,10,...,2,751,0253,1,NaN,NaN,NaN,NaN,NaN,2025-10-08 00:00:00
531016,01,163784,2024,361606,2024,2024-12-23 00:00:00.0000000,99,FUNDIP,Fundo Municipal de Iluminação Pública,10,...,2,751,0253,1,NaN,NaN,NaN,NaN,NaN,2025-10-08 00:00:00


In [7]:
df_orcamento_r = df_orcamento_r.loc[df_orcamento_r['CÓDIGO_FUNÇÃO']=='16']
df_orcamento_r = df_orcamento_r.loc[df_orcamento_r['ANO_LIQUIDAÇÃO']=='2024']
df_orcamento_r

,COD_EMPRESA_PMSP,COD_EMPENHO,ANO_EMPENHO,CÓDIGO_NLP,ANO_LIQUIDAÇÃO,DATA_LIQUIDAÇÃO,CÓDIGO_ÓRGÃO,SIGLA_ÓRGÃO,DESCRIÇÃO_ÓRGÃO,CÓDIGO_UNIDADE,...,CÓDIGO_EXERCÍCIO_FONTE,CÓDIGO_DESTINAÇÃO_RECURSO,CÓDIGO_VÍNCULO_PMSP,CÓDIGO_TIPO_CRÉDITO_ORÇAMENTÁRIO,REGIÃO,SUBPREFEITURA,DISTRITO,TIPO_REGIONALIZAÇÃO,VALOR_DETALHAMENTO_AÇÃO,DATA_EXTRAÇÃO
38848,01,41426,2024,94288,2024,2024-04-05 00:00:00.0000000,07,FMD,Fundo Municipal de Desenvolvimento Social,10,...,1,759,1224,0,Oeste,Subprefeitura Butantã,Supra-Distrital,Despesa Regionalizável,531084.11,2025-10-08 00:00:00
38849,01,41426,2024,94290,2024,2024-04-05 00:00:00.0000000,07,FMD,Fundo Municipal de Desenvolvimento Social,10,...,1,759,1224,0,Oeste,Subprefeitura Butantã,Supra-Distrital,Despesa Regionalizável,299606.31,2025-10-08 00:00:00
38850,01,41426,2024,153646,2024,2024-06-11 00:00:00.0000000,07,FMD,Fundo Municipal de Desenvolvimento Social,10,...,1,759,1224,0,Oeste,Subprefeitura Butantã,Supra-Distrital,Despesa Regionalizável,123846.15,2025-10-08 00:00:00
38851,01,41426,2024,190100,2024,2024-07-18 00:00:00.0000000,07,FMD,Fundo Municipal de Desenvolvimento Social,10,...,1,759,1224,0,Oeste,Subprefeitura Butantã,Supra-Distrital,Despesa Regionalizável,186592.28,2025-10-08 00:00:00
38852,01,41426,2024,276444,2024,2024-10-07 00:00:00.0000000,07,FMD,Fundo Municipal de Desenvolvimento Social,10,...,1,759,1224,0,Oeste,Subprefeitura Butantã,Supra-Distrital,Despesa Regionalizável,233551.72,2025-10-08 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
529500,01,80616,2024,272288,2024,2024-10-01 00:00:00.0000000,98,FUNDURB,Fundo de Desenvolvimento Urbano,14,...,2,759,0402,1,Sul,Subprefeitura Parelheiros,Supra-Distrital,Despesa Regionalizável,324656.75,2025-10-08 00:00:00
529501,01,80617,2024,210865,2024,2024-08-06 00:00:00.0000000,98,FUNDURB,Fundo de Desenvolvimento Urbano,14,...,2,759,0402,1,Sul,Subprefeitura Capela do Socorro,Supra-Distrital,Despesa Regionalizável,2578610.44,2025-10-08 00:00:00
529502,01,84580,2024,210866,2024,2024-08-06 00:00:00.0000000,98,FUNDURB,Fundo de Desenvolvimento Urbano,14,...,2,759,0402,1,Sul,Subprefeitura Capela do Socorro,Supra-Distrital,Despesa Regionalizável,499937.05,2025-10-08 00:00:00
529503,01,84580,2024,272023,2024,2024-10-01 00:00:00.0000000,98,FUNDURB,Fundo de Desenvolvimento Urbano,14,...,2,759,0402,1,Sul,Subprefeitura Capela do Socorro,Supra-Distrital,Despesa Regionalizável,78673.39,2025-10-08 00:00:00


Vamos conferir qual o percentual do orçamento está presente na tabela de detalhamento e regionalizado a nível de subprefeitura.

In [8]:
df_orcamento_r['VALOR_DETALHAMENTO_AÇÃO'].sum()/df_orcamento['Vl_Liquidado'].sum()

0.952678388166953

95,3% de detalhamento é um percentual excelente. Vamos checar a regionalização.

In [9]:
(
    df_orcamento_r
    .assign(regionalizado=df_orcamento_r['SUBPREFEITURA'].str.contains('Supra')==False)
    .groupby('regionalizado')
    ['VALOR_DETALHAMENTO_AÇÃO'].sum()/df_orcamento['Vl_Liquidado'].sum()
)

regionalizado
False    0.308241
True     0.644437
Name: VALOR_DETALHAMENTO_AÇÃO, dtype: object

O percentual de regionalização é de 64,4%. Menor, mas ainda parece ser satisfatório. Vamos manter as informações de regionalização também.

## Cumprimento da Meta 12: Prover 49.000 moradias de interesse social

Esse indicador está disponível de forma regionalizada na página do PdM da Secretaria de Governo Municipal. 

In [10]:
url_meta_12 = 'https://prefeitura.sp.gov.br/documents/d/planejamento/relatorio-de-final-do-programa-de-metas-2021-2024_marco2025-xlsx'

resp_meta_12 = re.get(url_meta_12)
resp_meta_12.raise_for_status()
xlsx_meta_12 = BytesIO(resp_meta_12.content)
excel_meta_12 = pd.ExcelFile(xlsx_meta_12)

excel_meta_12.sheet_names

['execução', 'regionalização', 'orçamento']

In [11]:
sheet_name = 'regionalização'
df_meta_12 = pd.read_excel(xlsx_meta_12,
                   sheet_name=sheet_name)
df_meta_12

,Número da Meta,Descrição,Área,Subprefeitura,2021,2022,2023,2024
0,2,Implantar o Prontuário Eletrônico em 100% das ...,Saúde,Aricanduva,44.0,44.0,100.0,100.0
1,2,Implantar o Prontuário Eletrônico em 100% das ...,Saúde,Butantã,0.0,0.0,0.0,100.0
2,2,Implantar o Prontuário Eletrônico em 100% das ...,Saúde,Campo Limpo,100.0,100.0,100.0,100.0
3,2,Implantar o Prontuário Eletrônico em 100% das ...,Saúde,Capela do Socorro,44.0,44.0,92.0,100.0
4,2,Implantar o Prontuário Eletrônico em 100% das ...,Saúde,Casa Verde,54.0,69.0,100.0,100.0
...,...,...,...,...,...,...,...,...
960,84,Implantar quatro unidades da Escola Municipal ...,Cultura,Butantã,0.0,0.0,1.0,1.0
961,84,Implantar quatro unidades da Escola Municipal ...,Cultura,Freguesia/Brasilândia,0.0,1.0,1.0,1.0
962,84,Implantar quatro unidades da Escola Municipal ...,Cultura,Itaim Paulista,0.0,0.0,1.0,1.0
963,84,Implantar quatro unidades da Escola Municipal ...,Cultura,Parelheiros,0.0,0.0,0.0,1.0


In [12]:
df_meta_12 = df_meta_12[df_meta_12['Número da Meta']==12]
df_meta_12

,Número da Meta,Descrição,Área,Subprefeitura,2021,2022,2023,2024
151,12,Prover 49.000 moradias de interesse social.,Habitação,Butantã,216.0,546.0,3081.0,7261.0
152,12,Prover 49.000 moradias de interesse social.,Habitação,Campo Limpo,1316.0,1748.0,1748.0,2161.0
153,12,Prover 49.000 moradias de interesse social.,Habitação,Capela do Socorro,562.0,2040.0,2124.0,2140.0
154,12,Prover 49.000 moradias de interesse social.,Habitação,Casa Verde,0.0,108.0,128.0,128.0
155,12,Prover 49.000 moradias de interesse social.,Habitação,Cidade Ademar,0.0,8.0,78.0,78.0
156,12,Prover 49.000 moradias de interesse social.,Habitação,Cidade Tiradentes,50.0,50.0,347.0,507.0
157,12,Prover 49.000 moradias de interesse social.,Habitação,Ermelino Matarazzo,0.0,0.0,0.0,266.0
158,12,Prover 49.000 moradias de interesse social.,Habitação,Freguesia/Brasilândia,228.0,228.0,228.0,502.0
159,12,Prover 49.000 moradias de interesse social.,Habitação,Guaianases,0.0,968.0,1122.0,1322.0
160,12,Prover 49.000 moradias de interesse social.,Habitação,Ipiranga,1694.0,1965.0,2205.0,2685.0


## Produção de habitação de interesse social

O indicador do formulário 19 não está ligado diretamente a nenhuma meta específica do PdM, mas está em consonância com o indicador 11.01.03 dos ODS da Agenda 2030. Os dados desse indicador estão disponíveis no [Observasampa](https://observasampa.prefeitura.sp.gov.br/).

In [13]:
url_his = 'https://observasampa.prefeitura.sp.gov.br/arquivo.php?cd_indicador=285'
df_his = pd.read_csv(url_his,
                     sep=';',
                     decimal=',',
                     encoding='utf8')
df_his

,Unnamed: 0,região,nivel_regional,indicador,2017,2018,2019,2020,2021,2022,2023,2024
0,0,São Paulo,Município,11.01.03 Número de Unidades Habitacionais entr...,1982.0,2263.0,5153.0,5697.0,3282.0,2400.0,755.0,1145.0
1,0,Pirituba-Jaraguá,Subprefeitura,11.01.03 Número de Unidades Habitacionais entr...,NaN,NaN,NaN,592.0,0.0,0.0,0.0,5.0
2,1,Sé,Subprefeitura,11.01.03 Número de Unidades Habitacionais entr...,NaN,NaN,NaN,0.0,210.0,417.0,0.0,190.0
3,2,Vila Mariana,Subprefeitura,11.01.03 Número de Unidades Habitacionais entr...,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
4,3,Vila Maria-Vila Guilherme,Subprefeitura,11.01.03 Número de Unidades Habitacionais entr...,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
124,91,Sapopemba,Distrito,11.01.03 Número de Unidades Habitacionais entr...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0
125,92,Saúde,Distrito,11.01.03 Número de Unidades Habitacionais entr...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0
126,93,Sé,Distrito,11.01.03 Número de Unidades Habitacionais entr...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0
127,94,Socorro,Distrito,11.01.03 Número de Unidades Habitacionais entr...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0


In [14]:
df_his = df_his[df_his['nivel_regional']=='Subprefeitura']
df_his

,Unnamed: 0,região,nivel_regional,indicador,2017,2018,2019,2020,2021,2022,2023,2024
1,0,Pirituba-Jaraguá,Subprefeitura,11.01.03 Número de Unidades Habitacionais entr...,NaN,NaN,NaN,592.0,0.0,0.0,0.0,5.0
2,1,Sé,Subprefeitura,11.01.03 Número de Unidades Habitacionais entr...,NaN,NaN,NaN,0.0,210.0,417.0,0.0,190.0
3,2,Vila Mariana,Subprefeitura,11.01.03 Número de Unidades Habitacionais entr...,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
4,3,Vila Maria-Vila Guilherme,Subprefeitura,11.01.03 Número de Unidades Habitacionais entr...,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
5,4,Vila Prudente,Subprefeitura,11.01.03 Número de Unidades Habitacionais entr...,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
6,5,Itaquera,Subprefeitura,11.01.03 Número de Unidades Habitacionais entr...,NaN,NaN,NaN,0.0,0.0,0.0,0.0,5.0
7,6,Jabaquara,Subprefeitura,11.01.03 Número de Unidades Habitacionais entr...,NaN,NaN,NaN,308.0,0.0,0.0,0.0,0.0
8,7,São Miguel,Subprefeitura,11.01.03 Número de Unidades Habitacionais entr...,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
9,8,Sapopemba,Subprefeitura,11.01.03 Número de Unidades Habitacionais entr...,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
10,9,Itaim Paulista,Subprefeitura,11.01.03 Número de Unidades Habitacionais entr...,NaN,NaN,NaN,965.0,600.0,97.0,0.0,1.0


In [15]:
df_his = df_his.loc[:, [
    'região',
    'indicador',
    '2021',
    '2022',
    '2023'
]]

df_his

,região,indicador,2021,2022,2023
1,Pirituba-Jaraguá,11.01.03 Número de Unidades Habitacionais entr...,0.0,0.0,0.0
2,Sé,11.01.03 Número de Unidades Habitacionais entr...,210.0,417.0,0.0
3,Vila Mariana,11.01.03 Número de Unidades Habitacionais entr...,0.0,0.0,0.0
4,Vila Maria-Vila Guilherme,11.01.03 Número de Unidades Habitacionais entr...,0.0,0.0,0.0
5,Vila Prudente,11.01.03 Número de Unidades Habitacionais entr...,0.0,0.0,0.0
6,Itaquera,11.01.03 Número de Unidades Habitacionais entr...,0.0,0.0,0.0
7,Jabaquara,11.01.03 Número de Unidades Habitacionais entr...,0.0,0.0,0.0
8,São Miguel,11.01.03 Número de Unidades Habitacionais entr...,0.0,0.0,0.0
9,Sapopemba,11.01.03 Número de Unidades Habitacionais entr...,0.0,0.0,0.0
10,Itaim Paulista,11.01.03 Número de Unidades Habitacionais entr...,600.0,97.0,0.0


In [16]:
df_his = df_his.melt(
        ['região', 'indicador'],
        ['2021', '2022', '2023'],
        'ano',
        'qtd_unidades'
    )

df_his

,região,indicador,ano,qtd_unidades
0,Pirituba-Jaraguá,11.01.03 Número de Unidades Habitacionais entr...,2021,0.0
1,Sé,11.01.03 Número de Unidades Habitacionais entr...,2021,210.0
2,Vila Mariana,11.01.03 Número de Unidades Habitacionais entr...,2021,0.0
3,Vila Maria-Vila Guilherme,11.01.03 Número de Unidades Habitacionais entr...,2021,0.0
4,Vila Prudente,11.01.03 Número de Unidades Habitacionais entr...,2021,0.0
...,...,...,...,...
91,Santana-Tucuruvi,11.01.03 Número de Unidades Habitacionais entr...,2023,0.0
92,Santo Amaro,11.01.03 Número de Unidades Habitacionais entr...,2023,0.0
93,São Mateus,11.01.03 Número de Unidades Habitacionais entr...,2023,0.0
94,Butantã,11.01.03 Número de Unidades Habitacionais entr...,2023,122.0


## Número de termos de Permissão de Uso (TPU) emitidos em nome da mulher da familia

Esse indicador possui é um exemplo da transversalidade de gênero, que integra a perspectiva de gênero na construção da politica pública voltada à provisão de habitação de interesse social. Os dados desse indicador estão disponíveis, de maneira não regionalizada, no [Observasampa](https://observasampa.prefeitura.sp.gov.br/), no indicador 05.0a.04. Posteriormente, os dados regionalizados estarão presentes no relatório da Função Habitação.

In [17]:
url_tpu = 'https://observasampa.prefeitura.sp.gov.br/arquivo.php?cd_indicador=662'
df_tpu = pd.read_csv(url_tpu,
                     sep=';',
                     decimal=',',
                     encoding='utf8')
df_tpu

,Unnamed: 0,região,nivel_regional,indicador,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,0,São Paulo,Município,05.0a.04 Número de termos de Permissão de Uso ...,327.0,234.0,386.0,544.0,192.0,300.0,285.0,293.0,384.0,458.0


In [18]:
df_tpu = df_tpu.loc[:, [
    'indicador',
    '2021',
    '2022',
    '2023'
]]

df_tpu

,indicador,2021,2022,2023
0,05.0a.04 Número de termos de Permissão de Uso ...,285.0,293.0,384.0


In [19]:
df_tpu = df_tpu.melt(
        ['indicador'],
        ['2021', '2022', '2023'],
        'ano',
        'qtd_termos'
    )

df_tpu

,indicador,ano,qtd_termos
0,05.0a.04 Número de termos de Permissão de Uso ...,2021,285.0
1,05.0a.04 Número de termos de Permissão de Uso ...,2022,293.0
2,05.0a.04 Número de termos de Permissão de Uso ...,2023,384.0


## Número de domicílios em favelas

Para utilizarmos os dados de favelas e comunidades urbanas do último censo, primeiro precisamos extrair os dados geográficos do site do IBGE. Como o arquivo contém todas as favelas do Brasil, vamos filtrar apenas as favelas do município de São Paulo.

In [20]:
url = "https://ftp.ibge.gov.br/Censos/Censo_Demografico_2022/Favelas_e_comunidades_urbanas_Resultados_do_universo/arquivos_vetoriais/poligonos_FCUs_shp.zip"
df_favelas = gpd.read_file(url)
df_favelas = df_favelas[df_favelas['cd_mun']=='3550308']
df_favelas

,cd_fcu,nm_fcu,cd_uf,nm_uf,sigla_uf,cd_mun,nm_mun,geometry
0,35503080069,Jardim Miliunas,35,São Paulo,SP,3550308,São Paulo,"POLYGON ((-46.37112 -23.5, -46.37141 -23.5001,..."
8,35503082222,Avenida Arraias do Araguaia,35,São Paulo,SP,3550308,São Paulo,"POLYGON ((-46.50383 -23.58049, -46.50361 -23.5..."
22,35503081646,Pavanas,35,São Paulo,SP,3550308,São Paulo,"POLYGON ((-46.64777 -23.66911, -46.6481 -23.66..."
34,35503081067,Jardim São Carlos / Gleba 2,35,São Paulo,SP,3550308,São Paulo,"POLYGON ((-46.41058 -23.55835, -46.41067 -23.5..."
67,35503081121,Jardim Jaraguá II,35,São Paulo,SP,3550308,São Paulo,"POLYGON ((-46.79134 -23.45316, -46.7916 -23.45..."
...,...,...,...,...,...,...,...,...
12316,35503080507,São Francisco,35,São Paulo,SP,3550308,São Paulo,"POLYGON ((-46.64645 -23.70244, -46.64651 -23.7..."
12319,35503080372,Jardim das Rosas,35,São Paulo,SP,3550308,São Paulo,"POLYGON ((-46.78857 -23.66081, -46.78875 -23.6..."
12320,35503080334,Souza Dantas,35,São Paulo,SP,3550308,São Paulo,"POLYGON ((-46.66071 -23.64736, -46.66089 -23.6..."
12321,35503081241,Sete de Setembro II,35,São Paulo,SP,3550308,São Paulo,"POLYGON ((-46.6869 -23.77295, -46.6867 -23.773..."


Em seguida, vamos conferir o CRS utilizado no arquivo e, caso necessário, converter para o EPSG 31983.

In [21]:
df_favelas.crs

<Geographic 2D CRS: EPSG:4674>
Name: SIRGAS 2000
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: Latin America - Central America and South America - onshore and offshore. Brazil - onshore and offshore.
- bounds: (-122.19, -59.87, -25.28, 32.72)
Datum: Sistema de Referencia Geocentrico para las AmericaS 2000
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [22]:
df_favelas = df_favelas.to_crs(epsg=31983)
df_favelas

,cd_fcu,nm_fcu,cd_uf,nm_uf,sigla_uf,cd_mun,nm_mun,geometry
0,35503080069,Jardim Miliunas,35,São Paulo,SP,3550308,São Paulo,"POLYGON ((359999.294 7400461.018, 359969.451 7..."
8,35503082222,Avenida Arraias do Araguaia,35,São Paulo,SP,3550308,São Paulo,"POLYGON ((346540.261 7391411.884, 346562.181 7..."
22,35503081646,Pavanas,35,São Paulo,SP,3550308,São Paulo,"POLYGON ((331961.694 7381437.08, 331927.976 73..."
34,35503081067,Jardim São Carlos / Gleba 2,35,São Paulo,SP,3550308,São Paulo,"POLYGON ((356033.213 7393960.866, 356024.025 7..."
67,35503081121,Jardim Jaraguá II,35,São Paulo,SP,3550308,São Paulo,"POLYGON ((317018.47 7405175.928, 316992.312 74..."
...,...,...,...,...,...,...,...,...
12316,35503080507,São Francisco,35,São Paulo,SP,3550308,São Paulo,"POLYGON ((332139.677 7377746.652, 332133.391 7..."
12319,35503080372,Jardim das Rosas,35,São Paulo,SP,3550308,São Paulo,"POLYGON ((317588.774 7382183.16, 317570.598 73..."
12320,35503080334,Souza Dantas,35,São Paulo,SP,3550308,São Paulo,"POLYGON ((330614.329 7383829.698, 330595.928 7..."
12321,35503081241,Sete de Setembro II,35,São Paulo,SP,3550308,São Paulo,"POLYGON ((328106.645 7369890.12, 328127.518 73..."


Agora, vemos também que os atributos com a quantidade de domicílios não estão presentes no arquivo. Portanto, precisamos baixar o arquivo de atributos, que está disponível no mesmo site do IBGE.

In [23]:
url = 'https://sidra.ibge.gov.br/geratabela?format=xlsx&name=tabela9887_35_FAV_COM.xlsx&terr=N&rank=-&query=t/9887/n170/in%20n3%2035/v/allxp/p/all/c3/all/l/v,p%2Bc3,t'
df_favelas_domicilios = pd.read_excel(url,
                                      skiprows=4,
                                      engine='openpyxl',
                                      dtype=str)
df_favelas_domicilios

,Unnamed: 0,Total,Particular,Particular permanente,Particular permanente ocupado,Particular permanente ocupado - com entrevista,Particular permanente ocupado - sem entrevista,Particular permanente não ocupado,Particular permanente não ocupado - vago,Particular permanente não ocupado - uso ocasional,Particular improvisado,Coletivo,Coletivo - com morador,Coletivo - sem morador
0,Zincão - Americana (SP),174,174,174,163,160,3,11,11,-,-,-,-,-
1,Planalto da Serra - Amparo (SP),145,144,144,110,104,6,34,17,17,-,1,-,1
2,Núcleo Vicente Matheus - Arujá (SP),63,63,63,58,55,3,5,5,-,-,-,-,-
3,Antiga Tecnon - Arujá (SP),165,165,165,143,131,12,22,13,9,-,-,-,-
4,Invasão Área Verde (Área 4) - Arujá (SP),164,164,164,153,130,23,11,9,2,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3119,Tatiana A - Votorantim (SP),264,264,264,250,237,13,14,13,1,-,-,-,-
3120,Tatiana B - Votorantim (SP),138,138,32,22,18,4,10,10,-,106,-,-,-
3121,Rosa de Luxemburgo - Votorantim (SP),116,116,116,116,116,-,-,-,-,-,-,-,-
3122,Capoavinha - Votorantim (SP),107,107,107,107,104,3,-,-,-,-,-,-,-


O arquivo contém todas as favelas e comunidades urbanas do Estado de São Paulo, então vamos filtrar apenas as favelas do município de São Paulo.

In [24]:
df_favelas_domicilios = df_favelas_domicilios[df_favelas_domicilios.iloc[:,0].str.contains('São Paulo (SP)', regex=False)]
df_favelas_domicilios

,Unnamed: 0,Total,Particular,Particular permanente,Particular permanente ocupado,Particular permanente ocupado - com entrevista,Particular permanente ocupado - sem entrevista,Particular permanente não ocupado,Particular permanente não ocupado - vago,Particular permanente não ocupado - uso ocasional,Particular improvisado,Coletivo,Coletivo - com morador,Coletivo - sem morador
1573,Mirangoaba II - São Paulo (SP),281,281,281,268,259,9,13,10,3,-,-,-,-
1574,Dezenove - São Paulo (SP),335,335,335,307,298,9,28,27,1,-,-,-,-
1575,Jardim Cotinha - São Paulo (SP),337,337,336,327,296,31,9,8,1,1,-,-,-
1576,Sônia Ribeiro - São Paulo (SP),390,390,298,257,239,18,41,40,1,92,-,-,-
1577,Núcleo B - São Francisco Global - São Paulo (SP),1124,1124,1124,967,695,272,157,140,17,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2927,Rua Luís Mateus - São Paulo (SP),274,274,89,76,73,3,13,13,-,185,-,-,-
2928,Rua Go Sugaya - São Paulo (SP),71,71,71,43,18,25,28,27,1,-,-,-,-
2929,Pedro Alba - São Paulo (SP),101,101,101,90,85,5,11,11,-,-,-,-,-
2930,Rua Mato Grosso - São Paulo (SP),104,104,104,88,86,2,16,15,1,-,-,-,-


In [25]:
df_favelas_domicilios.shape[0] == df_favelas.shape[0]

True

Como as duas tabelas possuem a mesma quantidade de linhas, podemos assumir que o filtro foi realizado corretamente.

Vamos também garantir que as colunas de atributos estão todas com o formato de inteiro. Caso existam colunas com formato string, pode significar que existem valores especiais, conforme explicado na planilha "Notas" disponível no arquivo.

| Símbolo | Significado |
|---------|-------------|
| - | Zero absoluto, não resultante de um cálculo ou arredondamento. Ex: Em determinado município não existem pessoas de 14 anos de idade sem instrução. |
| 0 | Zero resultante de um cálculo ou arredondamento. Ex: A inflação do feijão em determinada Região Metropolitana foi 0. Determinado município produziu 400 kg de sementes de girassol e os dados da tabela são expressos em toneladas. |
| X | Valor inibido para não identificar o informante. Ex: Determinado município só possui uma empresa produtora de cimento, logo o valor de sua produção deve ser inibido. |
| .. | Valor não se aplica. Ex: Não se pode obter o total da produção agrícola em determinado município quando os produtos agrícolas são contabilizados com unidades de medida distintas. |
| ... | Valor não disponível. Ex: A produção de feijão em determinado município não foi pesquisada ou determinado município não existia no ano da pesquisa. |
| A a Z (exceto X) | Significa uma faixa de valores. Varia em função da tabela (se for o caso). Ex: O nível de precisão da produção estimada de combustíveis está na faixa A (95 a 100%) |

Primeiro, vamos substituir os valores `-` por `0`, para que a conversão para inteiro funcione corretamente. Caso existam outros valores especiais, como `X`, `..`, ou `...`, devemos avaliar a possibilidade de excluir essas colunas.

In [26]:
df_favelas_domicilios.dtypes

Unnamed: 0                                           object
Total                                                object
Particular                                           object
Particular permanente                                object
Particular permanente ocupado                        object
Particular permanente ocupado - com entrevista       object
Particular permanente ocupado - sem entrevista       object
Particular permanente não ocupado                    object
Particular permanente não ocupado - vago             object
Particular permanente não ocupado - uso ocasional    object
Particular improvisado                               object
Coletivo                                             object
Coletivo - com morador                               object
Coletivo - sem morador                               object
dtype: object

In [27]:
for col in df_favelas_domicilios.columns:
    if (df_favelas_domicilios[col].dtype == 'object' and col != df_favelas_domicilios.columns[0]):
        df_favelas_domicilios.loc[:, col] = df_favelas_domicilios.loc[:, col].str.replace('-', '0')
        if df_favelas_domicilios[col].str.contains(r'[^\d]', regex=True).any():
            print(f'Coluna {col} contém valores não numéricos além de "-"')

In [28]:
dtypes = {col: 'int64'
          for col in df_favelas_domicilios.columns
          if col != df_favelas_domicilios.columns[0]}

dtypes

{'Total': 'int64',
 'Particular': 'int64',
 'Particular permanente': 'int64',
 'Particular permanente ocupado': 'int64',
 'Particular permanente ocupado - com entrevista': 'int64',
 'Particular permanente ocupado - sem entrevista': 'int64',
 'Particular permanente não ocupado': 'int64',
 'Particular permanente não ocupado - vago': 'int64',
 'Particular permanente não ocupado - uso ocasional': 'int64',
 'Particular improvisado': 'int64',
 'Coletivo': 'int64',
 'Coletivo - com morador': 'int64',
 'Coletivo - sem morador': 'int64'}

In [29]:
df_favelas_domicilios = df_favelas_domicilios.astype(dtypes)
df_favelas_domicilios.dtypes

Unnamed: 0                                           object
Total                                                 int64
Particular                                            int64
Particular permanente                                 int64
Particular permanente ocupado                         int64
Particular permanente ocupado - com entrevista        int64
Particular permanente ocupado - sem entrevista        int64
Particular permanente não ocupado                     int64
Particular permanente não ocupado - vago              int64
Particular permanente não ocupado - uso ocasional     int64
Particular improvisado                                int64
Coletivo                                              int64
Coletivo - com morador                                int64
Coletivo - sem morador                                int64
dtype: object

Agora, podemos juntar as duas tabelas utilizando o campo de nome da favela/comunidade urbana.

Primeiro, vamos extrair apenas o nome da favela/comunidade urbana do campo de nome, que contém o nome do município também.

In [30]:
df_favelas_domicilios.iloc[:, 0] = df_favelas_domicilios.iloc[:, 0].str.replace(' - São Paulo (SP)', '', regex=False)
df_favelas_domicilios

,Unnamed: 0,Total,Particular,Particular permanente,Particular permanente ocupado,Particular permanente ocupado - com entrevista,Particular permanente ocupado - sem entrevista,Particular permanente não ocupado,Particular permanente não ocupado - vago,Particular permanente não ocupado - uso ocasional,Particular improvisado,Coletivo,Coletivo - com morador,Coletivo - sem morador
1573,Mirangoaba II,281,281,281,268,259,9,13,10,3,0,0,0,0
1574,Dezenove,335,335,335,307,298,9,28,27,1,0,0,0,0
1575,Jardim Cotinha,337,337,336,327,296,31,9,8,1,1,0,0,0
1576,Sônia Ribeiro,390,390,298,257,239,18,41,40,1,92,0,0,0
1577,Núcleo B - São Francisco Global,1124,1124,1124,967,695,272,157,140,17,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2927,Rua Luís Mateus,274,274,89,76,73,3,13,13,0,185,0,0,0
2928,Rua Go Sugaya,71,71,71,43,18,25,28,27,1,0,0,0,0
2929,Pedro Alba,101,101,101,90,85,5,11,11,0,0,0,0,0
2930,Rua Mato Grosso,104,104,104,88,86,2,16,15,1,0,0,0,0


In [31]:
df_favelas_domicilios.iloc[:, 0].describe() 

count                    1359
unique                   1359
top       Tijolinhos da Mooca
freq                        1
Name: Unnamed: 0, dtype: object

Vemos, também, que todos os nomes são únicos, então podemos prosseguir com o merge sem problemas. Vamos apenas substituir o caractere `´` por `'`, porque seus usos não estão completamente padronizados entre as duas tabelas.

In [32]:
df_favelas.loc[:, 'nm_fcu'] = df_favelas['nm_fcu'].str.replace('´', "'", regex=False)
df_favelas_domicilios.iloc[:, 0] = df_favelas_domicilios.iloc[:, 0].str.replace('´', "'", regex=False)

In [33]:
df_favelas = (
    df_favelas
    .merge(
        df_favelas_domicilios,
        left_on='nm_fcu',
        right_on=df_favelas_domicilios.columns[0],
        how='left',
        validate='1:1'
    )
    .drop(columns=[df_favelas_domicilios.columns[0]])
)

df_favelas

,cd_fcu,nm_fcu,cd_uf,nm_uf,sigla_uf,cd_mun,nm_mun,geometry,Total,Particular,...,Particular permanente ocupado,Particular permanente ocupado - com entrevista,Particular permanente ocupado - sem entrevista,Particular permanente não ocupado,Particular permanente não ocupado - vago,Particular permanente não ocupado - uso ocasional,Particular improvisado,Coletivo,Coletivo - com morador,Coletivo - sem morador
0,35503080069,Jardim Miliunas,35,São Paulo,SP,3550308,São Paulo,"POLYGON ((359999.294 7400461.018, 359969.451 7...",411,411,...,392,213,179,19,18,1,0,0,0,0
1,35503082222,Avenida Arraias do Araguaia,35,São Paulo,SP,3550308,São Paulo,"POLYGON ((346540.261 7391411.884, 346562.181 7...",133,133,...,18,17,1,13,11,2,102,0,0,0
2,35503081646,Pavanas,35,São Paulo,SP,3550308,São Paulo,"POLYGON ((331961.694 7381437.08, 331927.976 73...",58,58,...,55,50,5,3,3,0,0,0,0,0
3,35503081067,Jardim São Carlos / Gleba 2,35,São Paulo,SP,3550308,São Paulo,"POLYGON ((356033.213 7393960.866, 356024.025 7...",496,496,...,465,312,153,31,29,2,0,0,0,0
4,35503081121,Jardim Jaraguá II,35,São Paulo,SP,3550308,São Paulo,"POLYGON ((317018.47 7405175.928, 316992.312 74...",118,118,...,94,71,23,24,19,5,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1354,35503080507,São Francisco,35,São Paulo,SP,3550308,São Paulo,"POLYGON ((332139.677 7377746.652, 332133.391 7...",413,413,...,382,350,32,31,28,3,0,0,0,0
1355,35503080372,Jardim das Rosas,35,São Paulo,SP,3550308,São Paulo,"POLYGON ((317588.774 7382183.16, 317570.598 73...",445,444,...,405,357,48,39,37,2,0,1,0,1
1356,35503080334,Souza Dantas,35,São Paulo,SP,3550308,São Paulo,"POLYGON ((330614.329 7383829.698, 330595.928 7...",742,742,...,612,576,36,122,114,8,8,0,0,0
1357,35503081241,Sete de Setembro II,35,São Paulo,SP,3550308,São Paulo,"POLYGON ((328106.645 7369890.12, 328127.518 73...",461,461,...,431,398,33,30,26,4,0,0,0,0


In [34]:
df_favelas.isna().sum()

cd_fcu                                               0
nm_fcu                                               0
cd_uf                                                0
nm_uf                                                0
sigla_uf                                             0
cd_mun                                               0
nm_mun                                               0
geometry                                             0
Total                                                0
Particular                                           0
Particular permanente                                0
Particular permanente ocupado                        0
Particular permanente ocupado - com entrevista       0
Particular permanente ocupado - sem entrevista       0
Particular permanente não ocupado                    0
Particular permanente não ocupado - vago             0
Particular permanente não ocupado - uso ocasional    0
Particular improvisado                               0
Coletivo  

Todas os registros foram combinados corretamente, então podemos prosseguir para a exportação dos dados.

# Exportando os arquivos

Neste notebook, vamos apenas salvar os arquivos extraídos na pasta de entrada de dados.

In [35]:
output_dir = path.join('data', 'input', 'urbanismo')

if not path.exists(output_dir):
    makedirs(output_dir)

for c, df in [('orcamento_habitacao_original', df_orcamento),
               ('orcamento_regionalizado_habitacao_original', df_orcamento_r),
               ('pdm_meta_12_original', df_meta_12),
               ('his_entregue_original', df_his),
               ('tpu_emitido_original', df_tpu)
               ]:
    filename=path.join(output_dir, c)
    df.to_csv(f'{filename}.csv',
              sep=';',
              decimal=',',
              encoding='utf8',
              index=False
              )
    
df_favelas.to_file(path.join(output_dir, 'favelas_original.gpkg'), driver='GPKG')